### 아래 링크에서 많은 내용을 참고
링크 : https://wikidocs.net/94600

In [1]:
#필요 모듈 불러오기
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request

# from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# 훈련시킬 데이터 가져오기
# 기존 리뷰 긍부정 분석시 사용할 데이터
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")

('ratings_total.txt', <http.client.HTTPMessage at 0x1db081f1c70>)

### 본인은 다른 리뷰들을 더 크롤링하고 정리하여 더 많은 훈련데이터를 생성함
- 해당 데이터 생성시 다니던 회사의 리뷰 데이터도 이용했기 때문에 해당 데이터는 업로드 하지 않음

In [3]:
total_data = pd.read_excel("훈련데이터.xlsx")
total_data

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
...,...,...
321111,5,실밥이 하나 터졌는지 구멍이 나있어서 마음이 아팠지만 입어보고 ㅗㅜㅑ.. 이 정도는...
321112,5,옷 색감이고 재질이고 완전 좋아요!\n완전 제 스타일! 깔별로 소장각이에요 이건
321113,5,진짜 너무 마음에 들고 예뻐서 바로 샀지만 모든 컬러들이 예쁘지만 믹스라임이 되게 ...
321114,5,고민 하지말고 시간이 사세요 개이쁨


In [4]:
# 전처리
# total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
total_data['label'] = np.select([total_data.ratings > 3], [1], default=0)
total_data.drop_duplicates(subset=['reviews'], inplace=True) # reviews 열에서 중복인 내용이 있다면 중복 제거
train_data, test_data = train_test_split(total_data, test_size = 0.25, random_state = 42)
train_data['reviews'] = train_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# 한글과 공백을 제외하고 모두 제거
train_data['reviews'].replace('', np.nan, inplace=True)
test_data.drop_duplicates(subset = ['reviews'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['reviews'] = test_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['reviews'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

C:\Users\onrik\AppData\Local\Temp/ipykernel_20848/1437852413.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['reviews'] = train_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
C:\Users\onrik\AppData\Local\Temp/ipykernel_20848/1437852413.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['reviews'] = train_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update

전처리 후 테스트용 샘플의 개수 : 75500


In [5]:
# 사용자 개인사전 추가를 위해 해당 모듈 사용
from ckonlpy.tag import Twitter
Twitter = Twitter()
Twitter.add_dictionary("유통기한","Noun")
Twitter.add_dictionary("유통 기한","Noun")
Twitter.add_dictionary("사은품","Noun")
print(Twitter.morphs('와 이런 것도 상품이라고 차라리 내가 만드는 게 나을 뻔'))

C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['와', '이런', '것', '도', '상품', '이라고', '차라리', '내', '가', '만드는', '게', '나을', '뻔']


In [6]:
# 불용어 사전 가져오기
k_stopword = pd.read_csv("korean_stopword.csv")
stopwords = list(k_stopword['불용어'])
stopwords.append('어요')

In [7]:
from datetime import datetime
start = datetime.now()
train_data['reviews'] = train_data['reviews'].apply(lambda x: str(x))
train_data['tokenized'] = train_data['reviews'].apply(Twitter.morphs)
train_data['tokenized'] = train_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])
end = datetime.now()
end-start

C:\Users\onrik\AppData\Local\Temp/ipykernel_20848/4286763667.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['reviews'] = train_data['reviews'].apply(lambda x: str(x))
C:\Users\onrik\AppData\Local\Temp/ipykernel_20848/4286763667.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['tokenized'] = train_data['reviews'].apply(Twitter.morphs)
C:\Users\onrik\AppData\Local\Temp/ipykernel_20848/4286763667.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

datetime.timedelta(seconds=368, microseconds=549121)

In [8]:
from collections import Counter

In [9]:
negative_words = np.hstack(train_data[train_data.label == 0]['tokenized'].values)
positive_words = np.hstack(train_data[train_data.label == 1]['tokenized'].values)

In [10]:
start = datetime.now()
test_data['tokenized'] = test_data['reviews'].apply(Twitter.morphs)
test_data['tokenized'] = test_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])
end = datetime.now()
end-start

datetime.timedelta(seconds=125, microseconds=213299)

In [11]:
X_train = train_data['tokenized'].values
y_train = train_data['label'].values
X_test= test_data['tokenized'].values
y_test = test_data['label'].values

In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [13]:
threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 82457
등장 빈도가 1번 이하인 희귀 단어의 수: 42751
단어 집합에서 희귀 단어의 비율: 51.846416920334235
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.5508473776963254


In [14]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 39708


In [15]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [16]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

In [17]:
max_len = 80
below_threshold_len(max_len, X_train)

전체 샘플 중 길이가 80 이하인 샘플의 비율: 99.86404647138795


In [18]:
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

# 2. GRU를 이용한 분류

In [19]:
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [20]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(GRU(128))
model.add(Dense(1, activation='sigmoid'))

In [21]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [22]:
start = datetime.now()
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)
end = datetime.now()
end-start

Epoch 1/15
3021/3021 [==============================] - ETA: 0s - loss: 0.2318 - acc: 0.9120
Epoch 1: val_acc improved from -inf to 0.92249, saving model to best_model.h5
3021/3021 [==============================] - 236s 77ms/step - loss: 0.2318 - acc: 0.9120 - val_loss: 0.2057 - val_acc: 0.9225
Epoch 2/15
3021/3021 [==============================] - ETA: 0s - loss: 0.1905 - acc: 0.9307
Epoch 2: val_acc improved from 0.92249 to 0.92810, saving model to best_model.h5
3021/3021 [==============================] - 234s 77ms/step - loss: 0.1905 - acc: 0.9307 - val_loss: 0.1970 - val_acc: 0.9281
Epoch 3/15
3021/3021 [==============================] - ETA: 0s - loss: 0.1762 - acc: 0.9379
Epoch 3: val_acc improved from 0.92810 to 0.93006, saving model to best_model.h5
3021/3021 [==============================] - 234s 77ms/step - loss: 0.1762 - acc: 0.9379 - val_loss: 0.1912 - val_acc: 0.9301
Epoch 4/15
3020/3021 [============================>.] - ETA: 0s - loss: 0.1643 - acc: 0.9429
Epoch 4: v

datetime.timedelta(seconds=4107, microseconds=199710)

In [23]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

2360/2360 [==============================] - 29s 12ms/step - loss: 0.1956 - acc: 0.9294

 테스트 정확도: 0.9294


In [24]:
def sentiment_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = Twitter.morphs(new_sentence) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  
    score = float(loaded_model.predict(pad_new)) # 예측
    if(score > 0.5):
        return ("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
    else:
        return ("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))

In [25]:
removelist = ['(별점: 5점)','(별점: 4점)','(별점: 3점)','ok','보통','만족','좋아요','좋습니다']

### 긍부정을 분석할 리뷰 데이터를 가져오기
- 이미 긍부정을 판단하는 함수를 만들었기 때문에 아래 코드처럼 특정 데이터를 가져와서 원하는 방식으로 가공하면 됨

In [1]:
import pandas as pd 
ddf = pd.read_excel("리뷰_{}\\전채널 리뷰_{}.xlsx".format(datetime.strftime(datetime.now(),'%y%m%d'),datetime.strftime(datetime.now(),'%y%m%d')))
ddf['리뷰 분석'] = ddf['리뷰'].apply(lambda x: sentiment_predict(str(x)))
ddf['확률'] = ddf['리뷰 분석'].apply(lambda x: x[:6])
ddf['긍정 부정'] = ddf['리뷰 분석'].apply(lambda x: x[11:13])
ddf['퍼센트'] = ddf['확률'].apply(lambda x: float(x.split("%")[0]))
ddf = ddf.loc[(ddf['긍정 부정'] == '부정')|((ddf['긍정 부정'] == '긍정')&(ddf['퍼센트'] <= 60))]
ddf = ddf[~ddf.리뷰.isin(removelist)]
ddf = ddf.drop_duplicates(subset='리뷰')
ddf = ddf[['채널', '리뷰접수일', '상품번호', '주문번호', 'ID', '상품명', '상품', '리뷰', '확률','긍정 부정']]
brandlst = list(ddf.상품.unique())

with pd.ExcelWriter('리뷰 긍정부정 분석 결과_{}.xlsx'.format(datetime.now().strftime('%y%m%d'))) as writer:
    for brand in brandlst:
        branddf = ddf.loc[ddf['상품'] == brand]
        branddf.to_excel(writer, sheet_name = '{}'.format(brand), index = False)
        print(brand, ' 끝')